<a href="https://colab.research.google.com/github/AkhileshPandeyji/Udacity-deep-gocolab/blob/master/ConvolutionNeuralNet_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import math
from tensorflow.examples.tutorials.mnist import input_data
import PIL.Image as Image
import matplotlib.pyplot as plt

print(tf.__version__)

tf.logging.set_verbosity(tf.logging.ERROR)


1.14.0


In [0]:
# Initializations

#input layer
X = tf.placeholder(tf.float32,[None,28,28,1])

#we will not flatten this image becoz the positional 
# properties are lost.

# Known Labels
Y_ = tf.placeholder(tf.float32)

# percentage of dropouts
pkeep = tf.placeholder(tf.float32)

#constants
K = 32
L = 64
M = 1024
out = 10

#Weights and biases

W1 = tf.Variable(tf.truncated_normal([5,5,1,K],stddev=0.1))
B1 = tf.Variable(tf.ones([K])/10)

W2 = tf.Variable(tf.truncated_normal([5,5,K,L],stddev=0.1))
B2 = tf.Variable(tf.ones([L])/10)


W3 = tf.Variable(tf.truncated_normal([7*7*L,M],stddev=0.1))
B3 = tf.Variable(tf.zeros([M]))

W4 = tf.Variable(tf.truncated_normal([M,out],stddev=0.1))
B4 = tf.Variable(tf.zeros([out]))

In [0]:
# Building Model

def conv2d(x,W):
  return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def maxpool2d(x):
  return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')



Y1 = tf.nn.relu(conv2d(X,W1)+B1)
Y1 = maxpool2d(Y1)

Y2 = tf.nn.relu(conv2d(Y1,W2)+B2)
Y2 = maxpool2d(Y2)


Yf = tf.reshape(Y2,shape=[-1,7*7*L])

Y3 = tf.nn.relu(tf.matmul(Yf,W3)+B3)

Y3 = tf.nn.dropout(Y3,pkeep)

Ylogits = tf.matmul(Y3,W4)+B4
Y = tf.nn.softmax(Ylogits)

In [0]:
# Success Metrics = accuracy+loss+optimizer 

is_correct = tf.equal(tf.argmax(Y,1),tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))

#loss1 = -tf.reduce_sum(Y_*tf.log(Y))
loss2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_,logits=Ylogits))

optimizer = tf.train.AdamOptimizer(0.003)
#optimizer = tf.train.GradientDescentOptimizer(0.003)

train_step = optimizer.minimize(loss2)

In [5]:

BATCH_SIZE = 100

#getting datasets
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

num_train_examples = mnist.train.num_examples
num_val_examples = mnist.validation.num_examples
num_test_examples = mnist.test.num_examples

print(num_train_examples,num_val_examples,num_test_examples)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
55000 5000 10000


In [6]:
# Training = acccuracy on validation set

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for epoch in range(1000):
  batch_x,batch_y = mnist.train.next_batch(BATCH_SIZE)
  batch_x = np.reshape(batch_x,[-1,28,28,1])
  batch_y = np.reshape(batch_y,[-1,10])
  train_dict = {X:batch_x,Y_:batch_y,pkeep:0.75}
  sess.run(train_step,feed_dict=train_dict)
  
  if epoch % 100 == 0:
    
    t_a,t_l = sess.run([accuracy,loss2],feed_dict=train_dict)
    print("Training: Accuracy:{},Loss:{} .".format(t_a,t_l))
    
    val_x ,val_y= mnist.test.next_batch(BATCH_SIZE)
    val_x = np.reshape(val_x,[-1,28,28,1])
    val_y = np.reshape(val_y,[-1,10])
    
    v_a,v_l = sess.run([accuracy,loss2],feed_dict={X:val_x,Y_:val_y,pkeep:1.0})
    print("Validation: Accuracy:{},Loss:{} .".format(v_a,v_l))


Training: Accuracy:0.14000000059604645,Loss:21.935932159423828 .
Validation: Accuracy:0.15000000596046448,Loss:23.49658966064453 .
Training: Accuracy:0.9800000190734863,Loss:0.09300112724304199 .
Validation: Accuracy:0.9200000166893005,Loss:0.2115921974182129 .
Training: Accuracy:0.949999988079071,Loss:0.13091318309307098 .
Validation: Accuracy:0.949999988079071,Loss:0.17595186829566956 .
Training: Accuracy:0.9800000190734863,Loss:0.045380521565675735 .
Validation: Accuracy:0.9800000190734863,Loss:0.11655038595199585 .
Training: Accuracy:0.9800000190734863,Loss:0.07107169926166534 .
Validation: Accuracy:0.9700000286102295,Loss:0.09246206283569336 .
Training: Accuracy:0.9599999785423279,Loss:0.1285126805305481 .
Validation: Accuracy:0.9800000190734863,Loss:0.044687334448099136 .
Training: Accuracy:1.0,Loss:0.03754332289099693 .
Validation: Accuracy:0.9700000286102295,Loss:0.10976111143827438 .
Training: Accuracy:0.9800000190734863,Loss:0.06546539068222046 .
Validation: Accuracy:0.990000

In [7]:
# prediction
img_file = tf.keras.utils.get_file("test2cnn.png",origin="https://raw.githubusercontent.com/do-community/tensorflow-digit-recognition/master/test_img.png")
test_img = Image.open(img_file).convert('L')
test_img

In [8]:
test_img = np.invert(test_img).reshape(-1,28,28,1)
prediction = sess.run(tf.argmax(Y,1),feed_dict={X:test_img,pkeep:1.0})
print(np.squeeze(prediction))

2


In [9]:
# testing
for epoch in range(100):
  test_x,test_y = mnist.test.next_batch(BATCH_SIZE)
  test_x = np.reshape(test_x,[-1,28,28,1])
  test_y = np.reshape(test_y,[-1,10])
  
  test_a,test_l = sess.run([accuracy,loss2],feed_dict={X:test_x,Y_:test_y,pkeep:1.0})
  
  print("Testing: Accuracy:{},loss:{} .".format(test_a,test_l))

Testing: Accuracy:0.9700000286102295,loss:0.09832113236188889 .
Testing: Accuracy:0.9700000286102295,loss:0.09400168061256409 .
Testing: Accuracy:0.9900000095367432,loss:0.024183427914977074 .
Testing: Accuracy:0.9700000286102295,loss:0.13047100603580475 .
Testing: Accuracy:0.9800000190734863,loss:0.03769208863377571 .
Testing: Accuracy:0.9900000095367432,loss:0.03231829032301903 .
Testing: Accuracy:1.0,loss:0.030054187402129173 .
Testing: Accuracy:0.9900000095367432,loss:0.026706399396061897 .
Testing: Accuracy:0.9900000095367432,loss:0.030702048912644386 .
Testing: Accuracy:0.9900000095367432,loss:0.030165722593665123 .
Testing: Accuracy:0.9900000095367432,loss:0.02389710582792759 .
Testing: Accuracy:0.9700000286102295,loss:0.08162971585988998 .
Testing: Accuracy:0.9700000286102295,loss:0.05946320667862892 .
Testing: Accuracy:0.9800000190734863,loss:0.059244245290756226 .
Testing: Accuracy:0.9700000286102295,loss:0.08932279795408249 .
Testing: Accuracy:0.9700000286102295,loss:0.05581